# Data Cleaning

In [10]:
import numpy as np
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier

import warnings
warnings.filterwarnings('ignore')

In [2]:
yelp_df = pd.read_pickle('./yelp_details_df.pkl')

In [3]:
yelp_df.sample(3)

,id,alias,name,image_url,is_claimed,is_closed,url,phone,display_phone,review_count,...,rating,location,coordinates,photos,price,hours,transactions,messaging,special_hours,error
Castro Tarts,ExM3wD4Sn1LXsrBM5LXK0w,castro-tarts-san-francisco,Castro Tarts,https://s3-media3.fl.yelpcdn.com/bphoto/zKBqZ-...,True,False,https://www.yelp.com/biz/castro-tarts-san-fran...,+14158613268,(415) 861-3268,270.0,...,4.0,"{'address1': '564 Castro St', 'address2': '', ...","{'latitude': 37.75991, 'longitude': -122.43527}",[https://s3-media3.fl.yelpcdn.com/bphoto/zKBqZ...,$,"[{'open': [{'is_overnight': False, 'start': '0...","[pickup, delivery]",NaN,NaN,NaN
Sports Basement,z6Di_ZlR9YbwdFle7BVseg,sports-basement-san-francisco-4,Sports Basement,https://s3-media1.fl.yelpcdn.com/bphoto/8PObiF...,True,False,https://www.yelp.com/biz/sports-basement-san-f...,+14155753001,(415) 575-3001,945.0,...,4.5,"{'address1': '1590 Bryant St', 'address2': '',...","{'latitude': 37.7670181, 'longitude': -122.410...",[https://s3-media1.fl.yelpcdn.com/bphoto/8PObi...,$$,"[{'open': [{'is_overnight': False, 'start': '0...",[],NaN,NaN,NaN
New Jumbo Seafood Restaurant,_az0NrGR-v9P-xtKoE45CA,new-jumbo-seafood-restaurant-san-francisco,New Jumbo Seafood Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/UmWDXH...,True,False,https://www.yelp.com/biz/new-jumbo-seafood-res...,+14156811800,(415) 681-1800,207.0,...,2.5,"{'address1': '1532 Noriega St', 'address2': ''...","{'latitude': 37.75432, 'longitude': -122.48032}",[https://s3-media2.fl.yelpcdn.com/bphoto/UmWDX...,$$,"[{'open': [{'is_overnight': False, 'start': '1...",[],NaN,NaN,NaN


Load in the yelp business details and remove unecessary columns.

In [4]:
del yelp_df['coordinates']
del yelp_df['location']
del yelp_df['phone']
del yelp_df['is_claimed']

yelp_df.reset_index(inplace=True)

yelp_df = yelp_df[yelp_df.is_closed == False]
del yelp_df['is_closed']

Looking at the categories column, we can see its a list of dictionaries. I pull out the aliases and titles for each business into separate columns in the dataframe.

In [5]:
yelp_df.categories[0]

[{'alias': 'breakfast_brunch', 'title': 'Breakfast & Brunch'},
 {'alias': 'hawaiian', 'title': 'Hawaiian'},
 {'alias': 'asianfusion', 'title': 'Asian Fusion'}]

In [6]:
alias_list = []
title_list = []

for idx, row in yelp_df.iterrows():
    category_list = yelp_df.categories[idx]
    
    aliases = ''
    titles = ''
    
    for category in category_list:
        aliases = aliases + category['alias'] + ','
        titles = titles + category['title'] + ','
        
    alias_list.append(aliases)
    title_list.append(titles)
    
yelp_df['category_alias'] = alias_list
yelp_df['category_title'] = title_list

del yelp_df['categories']

Then I take the hours for each business and split them up into 7 columns with hours for each day of the week.

In [7]:
Su_hours_list = []
M_hours_list = []
T_hours_list = []
W_hours_list = []
Th_hours_list = []
F_hours_list = []
S_hours_list = []

for idx, row in yelp_df.iterrows():
    
    try:
        hours_dict = yelp_df.hours[idx][0]
        hours_list = hours_dict['open']
        
        Sun = 'closed'
        Mon = 'closed'
        Tues = 'closed'
        Wed = 'closed'
        Thurs = 'closed'
        Fri = 'closed'
        Sat = 'closed'

        for day in hours_list:
            hours = day['start'] + '-' + day['end']
            day_of_week = day['day']

            if day_of_week == 0:
                Sun = hours
            elif day_of_week == 1:
                Mon = hours
            elif day_of_week == 2:
                Tues = hours
            elif day_of_week == 3:
                Wed = hours
            elif day_of_week == 4:
                Thurs = hours
            elif day_of_week == 5:
                Fri = hours
            elif day_of_week == 6:
                Sat = hours
                
        Su_hours_list.append(Sun)
        M_hours_list.append(Mon)
        T_hours_list.append(Tues)
        W_hours_list.append(Wed)
        Th_hours_list.append(Thurs)
        F_hours_list.append(Fri)
        S_hours_list.append(Sat)
        
    except:
        Su_hours_list.append('None')
        M_hours_list.append('None')
        T_hours_list.append('None')
        W_hours_list.append('None')
        Th_hours_list.append('None')
        F_hours_list.append('None')
        S_hours_list.append('None')
        
yelp_df['sunday_hours'] = Su_hours_list
yelp_df['monday_hours'] = M_hours_list
yelp_df['tuesday_hours'] = T_hours_list
yelp_df['wednesday_hours'] = W_hours_list
yelp_df['thursday_hours'] = Th_hours_list
yelp_df['friday_hours'] = F_hours_list
yelp_df['saturday_hours'] = S_hours_list

Finally, I clean up the transactions column by removing all of the elements of the list of transactions for a business and making them into a string instead.

In [8]:
transactions = []

for idx, row in yelp_df.iterrows():
    transaction_list = yelp_df.transactions[idx]
    
    t = ''
    
    for transaction in transaction_list:
        t = t + transaction + ','
        
    transactions.append(t)
    
del yelp_df['hours']
del yelp_df['special_hours']
del yelp_df['messaging']
del yelp_df['transactions']

yelp_df['transactions'] = transactions

In [9]:
yelp_df.head()

,index,id,alias,name,image_url,url,display_phone,review_count,rating,photos,...,category_alias,category_title,sunday_hours,monday_hours,tuesday_hours,wednesday_hours,thursday_hours,friday_hours,saturday_hours,transactions
0,'āina,RRY9YSBKCuiRMA7M3qzV1w,āina-san-francisco-9,āina,https://s3-media2.fl.yelpcdn.com/bphoto/_QBWJ8...,https://www.yelp.com/biz/%C4%81ina-san-francis...,(415) 814-3815,939.0,4.0,[https://s3-media2.fl.yelpcdn.com/bphoto/_QBWJ...,...,"breakfast_brunch,hawaiian,asianfusion,","Breakfast & Brunch,Hawaiian,Asian Fusion,",closed,closed,1730-2200,1730-2200,1730-2200,1730-2200,closed,
1,100% Sweet Cafe,VPA1EFglGYDyodAWDm5-sA,100-sweet-cafe-san-francisco,100% Sweet Cafe,https://s3-media4.fl.yelpcdn.com/bphoto/LuOdA2...,https://www.yelp.com/biz/100-sweet-cafe-san-fr...,(415) 221-1628,532.0,3.0,[https://s3-media4.fl.yelpcdn.com/bphoto/LuOdA...,...,"asianfusion,desserts,chinese,","Asian Fusion,Desserts,Chinese,",1200-2300,1200-2300,1200-2300,1200-2300,1200-2300,1100-2300,1100-2300,
2,1275 café,X5L-KA4JMnxaXWjY8cikOQ,1275-cafe-san-francisco,1275 Cafe,,https://www.yelp.com/biz/1275-cafe-san-francis...,(415) 645-4585,1.0,4.0,[],...,"cafes,breakfast_brunch,","Cafes,Breakfast & Brunch,",1130-1400,1130-1400,1130-1400,1130-1400,0730-1100,1130-1400,1130-1400,
3,20th Salon and Barber,UbcuvklcY2Q38ZC07EUl_g,20th-salon-and-barber-san-francisco,20th Salon and Barber,https://s3-media4.fl.yelpcdn.com/bphoto/kv6hb2...,https://www.yelp.com/biz/20th-salon-and-barber...,(415) 824-1335,354.0,4.5,[https://s3-media4.fl.yelpcdn.com/bphoto/kv6hb...,...,"hair,barbers,","Hair Salons,Barbers,",closed,0900-2100,0900-2100,0900-2100,1100-2100,0900-2100,closed,
5,25 Lusk,zvIbFkcZwqhTuDF7t7eQsQ,twenty-five-lusk-san-francisco,Twenty Five Lusk,https://s3-media2.fl.yelpcdn.com/bphoto/cJ1J9E...,https://www.yelp.com/biz/twenty-five-lusk-san-...,(415) 495-5875,1373.0,4.0,[https://s3-media2.fl.yelpcdn.com/bphoto/cJ1J9...,...,"newamerican,cocktailbars,","American (New),Cocktail Bars,",1730-2200,1730-2200,1730-2200,1730-2200,1730-2300,1730-2300,1700-2100,"restaurant_reservation,"


In [11]:
yelp_df.to_pickle('./yelp_df_clean.pkl')
goog_df = pd.read_pickle('./goog_df.pkl')
review_vecs = pd.read_pickle('./review_vectors.pkl')

Now I want to combine all of my data. I pull in the google API business data and my yelp review topic vectors. Since I had some missing values in my dataset, I wanted to combine business data from both the Yelp Fusion API and the Google Places API to minimize missing values.

In [12]:
review_vecs.rename(columns={'index': 'business'}, inplace=True)

goog_and_vecs = pd.merge(left=goog_df, right=review_vecs, how='left', left_on='name', right_on='business')
df = pd.merge(left=goog_and_vecs, right=yelp_df, how='left', left_on='name', right_on='index')

In [13]:
df.head(3)

,name_x,price_level,types,rating_x,total_ratings,formatted_address,place_id,id_x,photo_links,photo_ref,...,category_alias,category_title,sunday_hours,monday_hours,tuesday_hours,wednesday_hours,thursday_hours,friday_hours,saturday_hours,transactions
0,Tory Burch,3.0,"clothing_store,store,point_of_interest,establi...",4.3,16,"845 Market St Suite 153, San Francisco, CA 941...",ChIJy4kpbIaAhYARno7p_1YeHEA,999c0d216088ec28a0ed20c22407d55936bf3ed8,"<a href=""https://maps.google.com/maps/contrib/...",CmRaAAAAAlM2n492V0ZT7rtTgEApKW7NSWyRTh2fC3r6ju...,...,"womenscloth,accessories,","Women's Clothing,Accessories,",1000-2030,1000-2030,1000-2030,1000-2030,1000-2030,1000-2030,1100-1900,
1,Tory Burch Flagship,3.0,"clothing_store,store,point_of_interest,establi...",4.2,10,"222 Stockton St, San Francisco, CA 94108, USA",ChIJY3nL-IiAhYAR6dJ1DRpjd20,299c07df51006de4400148acb5be671f7c8b6930,"<a href=""https://maps.google.com/maps/contrib/...",CmRaAAAAGt8PzrpcBABVB3MnAgtqsUsU4sM7as-jzsTcK7...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Split Pea Seduction,2.0,"meal_takeaway,store,restaurant,point_of_intere...",4.3,48,"138 6th St, San Francisco, CA 94103, USA",ChIJFQb9CYSAhYARzItJ85sh8eo,dd06d29ab2354aa7e0a8c55019ea10bfd8085fbf,"<a href=""https://maps.google.com/maps/contrib/...",CmRZAAAAT-hwryz06FPI1ehmvOtfoBrw9yKaINQ6rUuDkr...,...,"sandwiches,soup,salad,","Sandwiches,Soup,Salad,",0900-1600,0900-1600,0900-1600,0900-1600,0900-1600,closed,closed,"pickup,delivery,"


Next I will clean up some of the category columns that are rare and not useful for business comparison or were double counted when combining datasets.

In [14]:
cols_to_remove = ['accounting', 'airport', 'bus_station', 'campground', 'car_dealer', 'car_rental', 'car_repair', 
                  'car_wash', 'cemetery', 'colloquial_area', 'dentist', 'doctor', 'embassy', 'hospital',
                  'insurance_agency', 'intersection', 'lawyer', 'library', 'local_government_office', 'locality',
                  'locksmith', 'lodging', 'natural_feature', 'neighborhood', 'painter', 'parking', 'pharmacy',
                  'plumber', 'political', 'post_office', 'real_estate_agency', 'route', 'stadium', 'storage', 
                  'subpremise', 'subway_station', 'transit_station', 'travel_agency', 'university', 'veterinary_care']

for col in cols_to_remove:
    df = df[df[col] == 0]
    del df[col]

indices_to_remove = [21, 187, 203, 477, 511, 565, 679, 690, 897, 933, 946, 950, 1117, 1139, 1146, 1194, 1207, 1256, 
                     1327, 1337, 1364, 1365, 1425, 1489, 1520, 1521, 1541, 1624, 1638, 1661, 1681, 1694, 1717, 1719, 
                     1749, 1885, 1911, 1954, 1957, 1981, 2028, 2036, 2038, 2192, 2198, 2206, 2215, 2221, 2323, 2368,
                     2385, 2413, 2423, 2446, 2528, 2826, 3095, 3878, 4318, 4783, 4847, 4855, 4968, 5075, 5252, 5299, 
                     5318, 5491, 5648, 5681, 5742, 5916, 5925, 6005, 6051, 6076, 6090, 6126, 6134, 
                     6160, 6173, 6176, 6181, 6192, 6214, 6215, 6267]
drop_cols = ['premise', 'atm', 'bank', 'church', 'point_of_interest', 'place_of_worship', 'finance', 'gas_station',
            'general_contractor', 'park', 'school', 'category_title', 'types', 'name_y', 'error', 'compound_code', 
             'photo_ref', 'id_x', 'global_code', 'id_y']

for idx in indices_to_remove:
    try:
        df = df.drop(idx)
    except:
        pass
    
for col in drop_cols:
    del df[col]

Next I fill in missing price, rating, or review count values from the business taken from the Yelp Fusion API and fill them with price, rating, or review count values taken from the Google Places API. I then drop all of the uneccesary columns.

In [15]:
df['price'] = df['price'].fillna(df['price_level'])
df['rating_y'] = df['rating_y'].fillna(df['rating_x'])
df['review_count'] = df['review_count'].fillna(df['total_ratings'])

del df['price_level']
del df['rating_x']
del df['total_ratings']
del df['total']

I then get dummy variables for all of the categories listed for each business in the category_alias column. I concatenate the dummy variables to my dataframe and delete some columns that are not relevant for my recommender. 

In [16]:
df = pd.concat([df, df['category_alias'].str.get_dummies(sep=',')], axis=1)

df = df[df.advertising == 0]

del df['3dprinting']
del df['adoptionservices']
del df['adultedu']
del df['advertising']
del df['afghani']
del df['african']
del df['alternativemedicine']
del df['web_design']
del df['category_alias']

Then I loop through the categories and remove any column that appears in 5 or less businesses as this would not be a helpful category for recommendations. 

In [17]:
categories = ['aquarium','art_gallery','bakery','bar','beauty_salon','bicycle_store','book_store','bowling_alley',
              'cafe','city_hall','clothing_store','convenience_store','department_store','electronics_store',
              'establishment','florist','food','furniture_store','grocery_or_supermarket', 'gym','hair_care',
              'hardware_store','health', 'home_goods_store','jewelry_store','laundry','liquor_store','meal_delivery',
              'meal_takeaway','movie_rental','movie_theater','museum','night_club','painter','pet_store',
              'restaurant','shoe_store','shopping_mall','spa','store','supermarket','zoo']

didnt_work = []
for category in categories:
    try:
        count = df[category].value_counts()[1]
        if count <=5:
            del df[category]
    except:
        didnt_work.append(category)

I then changed the yelp dollar sign prices into numeric values.

In [18]:
df.price.replace('$', 1.0, inplace =True)
df.price.replace('$$', 2.0, inplace=True)
df.price.replace('$$$', 3.0, inplace=True)
df.price.replace('$$$$', 4.0, inplace=True)

I deleted some columns that were unhelpful for my recommender and combined category columns that were repeats of the same type of business.

In [19]:
del df['laundry']
del df['establishment']
del df['name_x']
del df['city_hall']

repeats = [('bar', 'bars'), ('bakery', 'bakeries'), ('bikes', 'bicycle_store'), ('bookstores', 'book_store'), 
          ('cafes', 'cafe'), ('convenience', 'convenience_store'), ('electronics', 'electronics_store'), 
          ('florist', 'florists'), ('furniture', 'furniture_store'), ('grocery', 'grocery_or_supermarket'), 
          ('gym', 'gyms'), ('hair', 'hair_care'), ('hardware', 'hardware_store'), ('homedecor', 'home_goods_store'), 
          ('hotdogs', 'hotdog'), ('jewelry', 'jewelry_store'), ('night_club', 'danceclubs'), ('petstore', 'pet_store'),
          ('shoes', 'shoe_store'), ('grocery', 'supermarket'), ('shopping_mall', 'shoppingcenters'), ('spas', 'spa')]

for pair in repeats:
    df[pair[0]] = df[pair[0]] + df[pair[1]]
    del df[pair[1]]
    
    df[pair[0]].replace(2,1, inplace=True)

In the next step, I split up the transactions column into food pickup, food delivery, and reservations columns.

In [20]:
df['reservations'] = 0

for idx, row in df.iterrows():
    trans_string = df.transactions[idx]
    try:
        trans_list = trans_string.split(',')

        for t in trans_list:
            if t == 'pickup':
                df.meal_takeaway[idx] = 1
            elif t == 'delivery':
                df.meal_delivery[idx] = 1
            elif t == 'restaurant_reservation':
                df.reservations[idx] = 1
    except:
        pass
    
del df['transactions']

And I make sure that my dataframe contains no null business name values.

In [21]:
df = df[df.business.notnull()]

Now that I have a clean dataframe, I want to take care of my missing price values by using KNN imputation. I start by creating a dataframe X that includes only numeric values.

In [22]:
X = df.drop(['business', 'formatted_address', 'place_id', 'photo_links', 'reviews', 'reviews_text', 'alias', 'index',
             'image_url', 'url', 'display_phone', 'photos', 'sunday_hours', 'monday_hours', 'tuesday_hours', 
             'wednesday_hours', 'thursday_hours', 'friday_hours', 'saturday_hours'], axis=1)

I split X into train and test sets where my test set is all of the rows with null price values.

In [23]:
X_train = X[X.price.notnull()]

y_train = X_train.price
X_train = X_train.drop(['price'], axis=1)

X_test = X[X.price.isnull()]
X_test = X_test.drop(['price'], axis=1)

X.price.value_counts()

2.0    2520
1.0    1272
3.0     489
4.0     145
Name: price, dtype: int64

I fit a KNN Classifier to my training set with k=5 nearest neighbors.

In [24]:
neighbors = KNeighborsClassifier(n_neighbors=5)
neighbors.fit(X_train, y_train) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

I predict the missing price values using the X_test set and save it to y_pred.

In [25]:
y_pred = neighbors.predict(X_test)

X_missing = X[X.price.isnull()]
X_missing.price = y_pred

Then I impute the missing price values in the dataframe with the KNN predicted values.

In [26]:
for idx, row in X_missing.iterrows():
    df.price[idx] = X_missing.price[idx]
    
df.price.value_counts()

2.0    2897
1.0    1477
3.0     525
4.0     148
Name: price, dtype: int64

Finally, I save both my full dataframe and a separate numbers only dataframe for my recommender calculations.

In [28]:
df.reset_index(inplace=True, drop=True)
df_only_nums = df.drop(['business', 'formatted_address', 'place_id', 'photo_links', 'reviews', 'alias', 'image_url', 
                       'url', 'display_phone', 'photos', 'sunday_hours', 'monday_hours', 'tuesday_hours', 
                       'wednesday_hours', 'thursday_hours', 'friday_hours', 'saturday_hours', 'price', 'lat', 'lng',
                       'review_count', 'rating'], axis=1)

In [29]:
df.to_pickle('./recs_df.pkl')
df_only_nums.to_pickle("./num_matrix.pkl")